In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Define the GRU class
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, bidirectional=True, dropout_val=0.2, num_directions=2, dim_out=2):
        super(GRU, self).__init__()
        self.hidden_dim, self.layer_dim, self.input_dim = hidden_dim, layer_dim, input_dim
        self.bidirectional = bidirectional
        self.num_directions = num_directions
        self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional=bidirectional, dropout=dropout_val)
        self.linear = nn.Linear(hidden_dim*num_directions, dim_out)
        self.act = nn.Tanh()

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim*self.num_directions, x.size(0), self.hidden_dim, device=x.device).requires_grad_()
        out, _ = self.gru(x, h0.detach())
        out = self.linear(out[:, -1, :])  # Get the last time step output for each sequence
        return self.act(out)

# Load and preprocess the data
data = pd.read_csv("/content/sample_data/vibration_data.csv")
data.dropna(inplace=True)
X = data[['ROLL_BAD', 'PITCH_BAD']].values
y = data[['ROLL_FILTERED', 'PITCH_FILTERED']].values

assert len(data) == 3017, "The dataset does not contain 3017 entries."

def create_sequences(X, y, sequence_length=30):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:i+sequence_length])
        y_seq.append(y[i+sequence_length])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, y)

X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.4, random_state=40)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=40)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled = scaler_X.transform(X_val_reshaped).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(X_test.shape)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

# Convert the data to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

# Create an instance of the GRU model
input_dim = 2
hidden_dim = 64
layer_dim = 2  # Multiple hidden layers
bidirectional = True
dropout_val = 0.2
num_directions = 2 if bidirectional else 1
dim_out = 2

model = GRU(input_dim, hidden_dim, layer_dim, bidirectional, dropout_val, num_directions, dim_out)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
batch_size = 30

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions on the test set
model.eval()
y_test_pred_tensor = model(X_test_tensor.to(device))

# Inverse transform the predictions
y_test_pred = scaler_y.inverse_transform(y_test_pred_tensor.cpu().detach().numpy())
y_test_actual = scaler_y.inverse_transform(y_test_tensor.cpu().detach().numpy())

# Calculate performance metrics
mse_test = mean_squared_error(y_test_actual, y_test_pred)
r_squared = r2_score(y_test_actual, y_test_pred)

print("Test Mean Squared Error:", mse_test)
print("R-squared:", r_squared)
print("The accuracy of our model is {:.2f}%".format(r_squared * 100))


Epoch [10/50], Loss: 0.5004
Epoch [20/50], Loss: 0.2716
Epoch [30/50], Loss: 0.2108
Epoch [40/50], Loss: 0.2042
Epoch [50/50], Loss: 0.1990
Test Mean Squared Error: 65.447334
R-squared: 0.794665906794404
The accuracy of our model is 79.47%


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Define the GRU class
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, bidirectional=True, dropout_val=0.2, num_directions=2, dim_out=2):
        super(GRU, self).__init__()
        self.hidden_dim, self.layer_dim, self.input_dim = hidden_dim, layer_dim, input_dim
        self.bidirectional = bidirectional
        self.num_directions = num_directions
        self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional=bidirectional, dropout=dropout_val)
        self.linear = nn.Linear(hidden_dim*num_directions, dim_out)
        self.act = nn.Tanh()

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim*self.num_directions, x.size(0), self.hidden_dim, device=x.device).requires_grad_()
        out, _ = self.gru(x, h0.detach())
        out = self.linear(out[:, -1, :])  # Get the last time step output for each sequence
        return self.act(out)

# Load and preprocess the data
data = pd.read_csv("/content/sample_data/Book1.csv")
data.dropna(inplace=True)
X = data[['ROLL_BAD', 'PITCH_BAD']].values
y = data[['ROLL_FILTERED', 'PITCH_FILTERED']].values

#assert len(data) == 3017, "The dataset does not contain 3017 entries."

def create_sequences(X, y, sequence_length=30):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:i+sequence_length])
        y_seq.append(y[i+sequence_length])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, y)

X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.4, random_state=40)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=40)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled = scaler_X.transform(X_val_reshaped).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(X_test.shape)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

# Convert the data to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

# Create an instance of the GRU model
input_dim = 2
hidden_dim = 64
layer_dim = 2  # Multiple hidden layers
bidirectional = True
dropout_val = 0.2
num_directions = 2 if bidirectional else 1
dim_out = 2

model = GRU(input_dim, hidden_dim, layer_dim, bidirectional, dropout_val, num_directions, dim_out)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
batch_size = 30

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions on the test set
model.eval()
y_test_pred_tensor = model(X_test_tensor.to(device))

# Inverse transform the predictions
y_test_pred = scaler_y.inverse_transform(y_test_pred_tensor.cpu().detach().numpy())
y_test_actual = scaler_y.inverse_transform(y_test_tensor.cpu().detach().numpy())

# Calculate performance metrics
mse_test = mean_squared_error(y_test_actual, y_test_pred)
r_squared = r2_score(y_test_actual, y_test_pred)

print("Test Mean Squared Error:", mse_test)
print("R-squared:", r_squared)
print("The accuracy of our model is {:.2f}%".format(r_squared * 100))


Epoch [10/50], Loss: 0.5774
Epoch [20/50], Loss: 0.4257
Epoch [30/50], Loss: 0.3964
Epoch [40/50], Loss: 0.3834
Epoch [50/50], Loss: 0.3751
Test Mean Squared Error: 272.48422
R-squared: 0.6248811488339931
The accuracy of our model is 62.49%


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Define the GRU class
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, bidirectional=True, dropout_val=0.2, num_directions=2, dim_out=2):
        super(GRU, self).__init__()
        self.hidden_dim, self.layer_dim, self.input_dim = hidden_dim, layer_dim, input_dim
        self.bidirectional = bidirectional
        self.num_directions = num_directions
        self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional=bidirectional, dropout=dropout_val)
        self.linear = nn.Linear(hidden_dim*num_directions, dim_out)
        self.act = nn.Tanh()

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim*self.num_directions, x.size(0), self.hidden_dim, device=x.device).requires_grad_()
        out, _ = self.gru(x, h0.detach())
        out = self.linear(out[:, -1, :])  # Get the last time step output for each sequence
        return self.act(out)

# Load and preprocess the data
data = pd.read_csv("/content/sample_data/fast_moving_data.csv")
data.dropna(inplace=True)
X = data[['ROLL_BAD', 'PITCH_BAD']].values
y = data[['ROLL_FILTERED', 'PITCH_FILTERED']].values

#assert len(data) == 3017, "The dataset does not contain 3017 entries."

def create_sequences(X, y, sequence_length=30):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:i+sequence_length])
        y_seq.append(y[i+sequence_length])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, y)

X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.4, random_state=40)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=40)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled = scaler_X.transform(X_val_reshaped).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(X_test.shape)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

# Convert the data to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

# Create an instance of the GRU model
input_dim = 2
hidden_dim = 64
layer_dim = 2  # Multiple hidden layers
bidirectional = True
dropout_val = 0.2
num_directions = 2 if bidirectional else 1
dim_out = 2

model = GRU(input_dim, hidden_dim, layer_dim, bidirectional, dropout_val, num_directions, dim_out)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
batch_size = 30

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions on the test set
model.eval()
y_test_pred_tensor = model(X_test_tensor.to(device))

# Inverse transform the predictions
y_test_pred = scaler_y.inverse_transform(y_test_pred_tensor.cpu().detach().numpy())
y_test_actual = scaler_y.inverse_transform(y_test_tensor.cpu().detach().numpy())

# Calculate performance metrics
mse_test = mean_squared_error(y_test_actual, y_test_pred)
r_squared = r2_score(y_test_actual, y_test_pred)

print("Test Mean Squared Error:", mse_test)
print("R-squared:", r_squared)
print("The accuracy of our model is {:.2f}%".format(r_squared * 100))


Epoch [10/50], Loss: 0.7089
Epoch [20/50], Loss: 0.6255
Epoch [30/50], Loss: 0.5929
Epoch [40/50], Loss: 0.5690
Epoch [50/50], Loss: 0.5464
Test Mean Squared Error: 1013.39136
R-squared: 0.45610156180184375
The accuracy of our model is 45.61%


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Define the GRU class
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, bidirectional=True, dropout_val=0.2, num_directions=2, dim_out=2):
        super(GRU, self).__init__()
        self.hidden_dim, self.layer_dim, self.input_dim = hidden_dim, layer_dim, input_dim
        self.bidirectional = bidirectional
        self.num_directions = num_directions
        self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True, bidirectional=bidirectional, dropout=dropout_val)
        self.linear = nn.Linear(hidden_dim*num_directions, dim_out)
        self.act = nn.Tanh()

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim*self.num_directions, x.size(0), self.hidden_dim, device=x.device).requires_grad_()
        out, _ = self.gru(x, h0.detach())
        out = self.linear(out[:, -1, :])  # Get the last time step output for each sequence
        return self.act(out)

# Load and preprocess the data
data = pd.read_csv("/content/sample_data/high_speed_data.csv")
data.dropna(inplace=True)
X = data[['ROLL_BAD', 'PITCH_BAD']].values
y = data[['ROLL_FILTERED', 'PITCH_FILTERED']].values

#assert len(data) == 3017, "The dataset does not contain 3017 entries."

def create_sequences(X, y, sequence_length=30):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:i+sequence_length])
        y_seq.append(y[i+sequence_length])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, y)

X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.4, random_state=40)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=40)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled = scaler_X.transform(X_val_reshaped).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(X_test.shape)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)

# Convert the data to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

# Create an instance of the GRU model
input_dim = 2
hidden_dim = 64
layer_dim = 2  # Multiple hidden layers
bidirectional = True
dropout_val = 0.2
num_directions = 2 if bidirectional else 1
dim_out = 2

model = GRU(input_dim, hidden_dim, layer_dim, bidirectional, dropout_val, num_directions, dim_out)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
batch_size = 30

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions on the test set
model.eval()
y_test_pred_tensor = model(X_test_tensor.to(device))

# Inverse transform the predictions
y_test_pred = scaler_y.inverse_transform(y_test_pred_tensor.cpu().detach().numpy())
y_test_actual = scaler_y.inverse_transform(y_test_tensor.cpu().detach().numpy())

# Calculate performance metrics
mse_test = mean_squared_error(y_test_actual, y_test_pred)
r_squared = r2_score(y_test_actual, y_test_pred)

print("Test Mean Squared Error:", mse_test)
print("R-squared:", r_squared)
print("The accuracy of our model is {:.2f}%".format(r_squared * 100))


Epoch [10/50], Loss: 0.8908
Epoch [20/50], Loss: 0.8665
Epoch [30/50], Loss: 0.8420
Epoch [40/50], Loss: 0.7979
Epoch [50/50], Loss: 0.7437
Test Mean Squared Error: 2979.0803
R-squared: 0.2703115550763192
The accuracy of our model is 27.03%
